<a href="https://colab.research.google.com/github/GillianYeung/gclyeung/blob/master/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fasttext
!pip install jieba

     |████████████████████████████████| 61kB 2.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2387958 sha256=53e3a0b28302827fbcf0bd7c205b4582f40809e87c026a630154c0b5cc92e966
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.zh_yue.vec

--2019-10-18 08:47:00--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.zh_yue.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130267704 (124M) [binary/octet-stream]
Saving to: ‘wiki.zh_yue.vec’

wiki.zh_yue.vec     100%[===================>] 124.23M  22.9MB/s    in 6.0s    

2019-10-18 08:47:07 (20.7 MB/s) - ‘wiki.zh_yue.vec’ saved [130267704/130267704]



In [0]:
import fasttext
import jieba
import jieba.analyse

In [4]:
jieba.load_userdict("user.dict")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.952 seconds.
Prefix dict has been built succesfully.


In [6]:
# Get stopwords list
def get_stopwords_list(file_path):
    file_stop = open(file_path, 'r')
    vec_out_words = []
    for szLine in file_stop:
        vec_out_words.append(szLine.strip())
    file_stop.close()
    return vec_out_words

# Segmentation without stop words
def sentence_seg(sentence):
    tags = "" if sentence is None else "/".join(jieba.cut(sentence))
    #print(tags)
    stopwords = get_stopwords_list('StopWords_V2.txt')
    start = 0
    end = 0
    curr_posn = 0
    out_str = ''
    for char in tags:
        if char == '/':
            word = tags[start:curr_posn]
            if word not in stopwords:
                out_str += word
                out_str += " "
            start = curr_posn + 1
        curr_posn += 1
    return out_str


with open('entireT_C.train', 'r', encoding = 'utf-8') as sourceFile, open('entireT_C2.train', 'a+', encoding = 'utf-8') as targetFile:
    for line in sourceFile:
        line_seg = sentence_seg(line)
        targetFile.write(line_seg)
        targetFile.write('\n')
    print('done！')

done！


In [0]:
model4=fasttext.train_supervised(input="entireT_C2.train",epoch=50,lr=0.5,minn=2,maxn=3,dim=300,pretrained_vectors='wiki.zh_yue.vec')

# **Testing**

In [8]:
#98 (original call guide)
script4 ="請留意如果您想提早償還全數貸款我哋會收取借款本金餘額及本金結欠的2%的款項如您未能按期償還全數每月還款我哋將會收取每次港幣400蚊嘅逾期還款手續費及按任何逾期未還嘅每月還款額收利息利息為月息2.25厘我們亦會參考對內對外信貸資料作最後審批 "
test4 = sentence_seg(script4)
print("test-#98: " + test4)
print ("test:",model4.predict(test4))
#98 (from transcript ivy 1s -1003)
test10 = sentence_seg("請留意返如果你想提早上船還數貸款我哋會收返你正本本金魚街同埋本間見面嘅兩個percent款項如果你未能按其實還全數每月還款我哋會生返你每次港幣400蚊嘅預期還款手續同埋我任何逾期未還嘅每月還款額收利息嘅你識唔會為原色二點至五呢我哋都會參考反對內對外嘅信貸資料在最後審批我所講嘅鍾小姐是否清楚明白同意 ")
print("test-#98 got it from Ivy 1s: " + test10)
print ("test:",model4.predict(test10))
#98 (from transcript ivy 0.7s - 1003)
script11 ="唔好明白咁呢度已經睇到㗎喇你可以下載返銅儲存返你啱啱行咗去文件咁請留返如果你想提早上還全數太高我哋會收返你借款本金魚夾同埋本間見面嘅兩個percent款項如果你未能按其實還全數每月還款我哋會收返你每次港幣400蚊嘅預期還款手續費同埋我任何逾期未還嘅每月還款額收利息嘅你 "
test11 = sentence_seg(script11)
print("test-#98 got it from Ivy 0.7s: " + test11)
print ("test:",model4.predict(test11))

test-#98: 請 留意 如果 想 提早償還 全數 貸款 會 收取 借款 本金 餘額 本金 結欠 2% 款項 如 未能 按期 償還 全數 每月 還款 將會 收取 每次 港幣 400 蚊 逾期 還款 手續費 按 任何 逾期未還 每月 還款 額收 利息 利息 為 月息 2.25 厘我們 亦 會 參考 對內對外 信貸資料 作 最 後 審批 
test: (('__label__R98',), array([0.99606836]))
test-#98 got it from Ivy 1s: 請 留意 返 如果 想 提早 上船 還數 貸款 會 收返 正本 本金 魚街 本間 見 面 兩個percent 款項 如果 未能 按其實 還全數 每月 還款 會 生返 每次 港幣 400 蚊 預 期 還款 手續 任何 逾期未還 每月 還款 額收 利息 識 唔會 為 原色 二點 至 五 會 參考 反 對內對外 信貸資料 在 最 後 審批 所 講 鍾 是否 清楚 明白 同意 
test: (('__label__R98',), array([0.92454273]))
test-#98 got it from Ivy 0.7s: 唔 好 明白 度 已經 睇 到 可以 下載 返銅 儲存 返 啱 啱 行 去 文件 請 留返 如果 想 提早 上 還全 數太高 會 收返 借款 本金 魚夾 本間 見 面 兩個percent 款項 如果 未能 按其實 還全數 每月 還款 會 收返 每次 港幣 400 蚊 預 期 還款 手續費 任何 逾期未還 每月 還款 額收 利息 
test: (('__label__R98',), array([0.81023252]))


In [9]:
test_irr = sentence_seg("雅虎香港新聞提供各類最新焦點及熱門新聞閱讀深入的相關報道新聞影片和圖片 ")
print ("test_case 1: " + test_irr)
print (model4.predict(test_irr))
test_irr3 = sentence_seg("首置800萬樓按九成可不符壓測經濟師增買家意欲二手樓年底料升半成 ")
print ("test_case 3: " + test_irr3)
print (model4.predict(test_irr3))
test_irr4 = sentence_seg("由2019年10月8日至11月24日憑滙豐信用卡累積本地合資格簽賬滿港幣6000元可享以下優惠 ")
print ("test_case : " + test_irr4)
print (model4.predict(test_irr4))
test_irr5 = sentence_seg("呢個 ")
print ("test_case : " + test_irr5)
print (model4.predict(test_irr5))
test_irr6 = sentence_seg("我哋 您嘅 ")
print ("test_case : " + test_irr6)
print (model4.predict(test_irr6))

test_case 1: 雅虎 香港 新聞 提供 各類 最新 焦點 熱門 新聞 閱讀 深入 相關 報道 新聞 影片 圖片 
(('__label__R129',), array([0.1098084]))
test_case 3: 首置 800 萬樓 按 九成 可 不符 壓測 經濟師 增買家 意欲 二手 樓 年底 料升 半成 
(('__label__R138',), array([0.06099574]))
test_case : 由 2019 年 10 月 8 日至 11 月 24 日 憑 滙豐 信用卡 累積 本地 合資格 簽賬 滿港幣 6000 元 可享 以下 優惠 
(('__label__R124',), array([0.13588201]))
test_case : 
(('__label__R188',), array([0.32313672]))
test_case :   
(('__label__R188',), array([0.32313672]))


# multiple T&**C**

In [11]:
testcase=sentence_seg("係呀你要唔開撳返開個超連結之後呢就撳返開始睇完之後撳返文件底破格仔剔返同埋近住就ok㗎喇咁呢度呢就寫一齊鍾小姐申請貸款之前請先考慮清楚財政狀況同埋玩風帆的你申請嘅分期買前呢銀碼會係呢個港幣嘅五萬蚊嘅咁就乜啦12個月還每月平息計零點二九零實際年利率六點五七號月功課係4310一個七全期總利息會係一千七百四十蚊嘅咁假如呢個銀碼呢覆核過冇問題之後呢咁")
print("testcase - multiple T&C (87/86/102): " + testcase)
print ("test:",model4.predict(testcase,k=5))
testcase1=sentence_seg("係呀你要唔開撳返開個超連結之後呢就撳返開始睇完之後撳返文件底破格仔剔返 ")
print("testcase1 - 102: " + testcase1)
print ("test:",model4.predict(testcase1,k=5))
testcase2=sentence_seg("同埋近住就ok㗎喇咁呢度呢就寫一齊鍾小姐申請貸款之前請先考慮清楚財政狀況 ")
print("testcase2 - 86: " + testcase2)
print ("test:",model4.predict(testcase2,k=5))
testcase3=sentence_seg("同埋玩風帆的你申請嘅分期買前呢銀碼會係呢個港幣嘅五萬蚊嘅咁就乜啦12個月還每月平息計零點二九零實際年利率六點五七號月功課係4310一個七全期總利息會係一千七百四十蚊嘅咁假如呢個銀碼呢覆核過冇問題之後呢咁 ")
print("testcase3 - 87: " + testcase3)
print ("test:",model4.predict(testcase3,k=5))
testcase4=sentence_seg("我哋會喺您尾數轉出到您所申請嘅貸款一旦貸款批核喺呢啲卡入面嘅加額優惠將自動失效您是否明白及同意假如覆核咗銀碼係無問題一經批核貸款會直接存入您指定嘅匯豐戶口但我哋唔會再有電話通知你啦我哋會寄出一封確認信清楚列明已批核嘅貸款額嘅詳細內容及第一期還款日期 ")
print("testcase4 - 88/89: " + testcase4)
print ("test:",model4.predict(testcase4,k=5))

testcase - multiple T&C (87/86/102): 係 呀 要 唔 開 撳 返 開個 超連結 之 後 撳 返 開始 睇 完 之 後 撳 返 文件 底 破格 仔 剔返 近住 ok 度 寫 一齊 鍾 申請 貸款 之前 請 先 考慮 清楚 財政 狀況 玩風帆 申請 分期 買前 銀碼 會 係 港 幣 五 萬 蚊 乜 12 個月還 每月 平息 計零點 二 九零 實際年利率 六點 五七 號月 功課 係 4310 一個 七 全期 總利息 會 係 一千七百 四十 蚊 假如 銀碼 覆核 過 冇 問題 之 後 
test: (('__label__R98', '__label__R138', '__label__R190', '__label__R86', '__label__R102'), array([0.08903697, 0.06555357, 0.0652247 , 0.06367951, 0.0598004 ]))
testcase1 - 102: 係 呀 要 唔 開 撳 返 開個 超連結 之 後 撳 返 開始 睇 完 之 後 撳 返 文件 底 破格 仔 剔返 
test: (('__label__R102', '__label__R97', '__label__R138', '__label__R140', '__label__R191'), array([0.50691754, 0.06267066, 0.05662801, 0.02646599, 0.02520175]))
testcase2 - 86: 近住 ok 度 寫 一齊 鍾 申請 貸款 之前 請 先 考慮 清楚 財政 狀況 
test: (('__label__R86', '__label__R88', '__label__R125', '__label__R190', '__label__R136'), array([0.88666761, 0.02098553, 0.01263096, 0.01238472, 0.00949346]))
testcase3 - 87: 玩風帆 申請 分期 買前 銀碼 會 係 港 幣 五 萬 蚊 乜 12 個月還 每月 平息 計零點 二 九零 實際年利率 六點 五七 號月 功課 係 4310 一個 七 全期 總利息 會 係 一千七百 四十 蚊 假如 

In [0]:
model6=fasttext.train_supervised(input="entireT_C2.train",epoch=100,lr=0.1,minn=2,maxn=3,dim=300,pretrained_vectors='wiki.zh_yue.vec',verbose=2,loss='ova')

In [17]:
testcase=sentence_seg("係呀你要唔開撳返開個超連結之後呢就撳返開始睇完之後撳返文件底破格仔剔返同埋近住就ok㗎喇咁呢度呢就寫一齊鍾小姐申請貸款之前請先考慮清楚財政狀況同埋玩風帆的你申請嘅分期買前呢銀碼會係呢個港幣嘅五萬蚊嘅咁就乜啦12個月還每月平息計零點二九零實際年利率六點五七號月功課係4310一個七全期總利息會係一千七百四十蚊嘅咁假如呢個銀碼呢覆核過冇問題之後呢咁")
print("testcase - multiple T&C (87/86/102): " + testcase)
print ("test:",model6.predict(testcase,k=5))
testcase1=sentence_seg("係呀你要唔開撳返開個超連結之後呢就撳返開始睇完之後撳返文件底破格仔剔返 ")
print("testcase1 - 102: " + testcase1)
print ("test:",model6.predict(testcase1,k=5))
testcase2=sentence_seg("同埋近住就ok㗎喇咁呢度呢就寫一齊鍾小姐申請貸款之前請先考慮清楚財政狀況 ")
print("testcase2 - 86: " + testcase2)
print ("test:",model6.predict(testcase2,k=5))
testcase3=sentence_seg("同埋玩風帆的你申請嘅分期買前呢銀碼會係呢個港幣嘅五萬蚊嘅咁就乜啦12個月還每月平息計零點二九零實際年利率六點五七號月功課係4310一個七全期總利息會係一千七百四十蚊嘅咁假如呢個銀碼呢覆核過冇問題之後呢咁 ")
print("testcase3 - 87: " + testcase3)
print ("test:",model6.predict(testcase3,k=5))
testcase4=sentence_seg("我哋會喺您尾數轉出到您所申請嘅貸款一旦貸款批核喺呢啲卡入面嘅加額優惠將自動失效您是否明白及同意假如覆核咗銀碼係無問題一經批核貸款會直接存入您指定嘅匯豐戶口但我哋唔會再有電話通知你啦我哋會寄出一封確認信清楚列明已批核嘅貸款額嘅詳細內容及第一期還款日期 ")
print("testcase4 - 88/89: " + testcase4)
print ("test:",model6.predict(testcase4,k=5))

testcase - multiple T&C (87/86/102): 係 呀 要 唔 開 撳 返 開個 超連結 之 後 撳 返 開始 睇 完 之 後 撳 返 文件 底 破格 仔 剔返 近住 ok 度 寫 一齊 鍾 申請 貸款 之前 請 先 考慮 清楚 財政 狀況 玩風帆 申請 分期 買前 銀碼 會 係 港 幣 五 萬 蚊 乜 12 個月還 每月 平息 計零點 二 九零 實際年利率 六點 五七 號月 功課 係 4310 一個 七 全期 總利息 會 係 一千七百 四十 蚊 假如 銀碼 覆核 過 冇 問題 之 後 
test: (('__label__R87', '__label__R102', '__label__R86', '__label__R98', '__label__R136'), array([0.00942259, 0.00832578, 0.00649827, 0.00349342, 0.00338661]))
testcase1 - 102: 係 呀 要 唔 開 撳 返 開個 超連結 之 後 撳 返 開始 睇 完 之 後 撳 返 文件 底 破格 仔 剔返 
test: (('__label__R102', '__label__R97', '__label__R124', '__label__R185', '__label__R178'), array([0.14034626, 0.02844604, 0.00913564, 0.00670285, 0.00670285]))
testcase2 - 86: 近住 ok 度 寫 一齊 鍾 申請 貸款 之前 請 先 考慮 清楚 財政 狀況 
test: (('__label__R86', '__label__R88', '__label__R180', '__label__R192', '__label__R187'), array([0.33459947, 0.00129431, 0.0010105 , 0.00094994, 0.00094994]))
testcase3 - 87: 玩風帆 申請 分期 買前 銀碼 會 係 港 幣 五 萬 蚊 乜 12 個月還 每月 平息 計零點 二 九零 實際年利率 六點 五七 號月 功課 係 4310 一個 七 全期 總利息 會 係 一千七百 四十 蚊 假如 銀